In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers = 2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers = 2)

In [73]:
image, label = train_data[0]

In [ ]:
image.size()

In [75]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [76]:
# Base class of a neural network
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__()

        # Define convolutional layer
        self.conv1 = nn.Conv2d(3, 12, 5) # 3 input channels, 12 feature maps, 5x5 size.
        self.pool = nn.MaxPool2d(2,2) # 4 pixels and creates one pixel (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # (14 - 5) / 1 = 9 + 1 = 10; (24, 10, 10) -> (24, 5, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [77]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum= 0.9)

In [ ]:
for epoch in range(50):
    print(f"Training epoch {epoch}...")

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Loss: {running_loss / len(train_loader):.4f}")


In [85]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

In [ ]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f"Accuracy: {accuracy}%")

In [ ]:
# Transform definition
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Function to load an image
def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

# Load all images from './assets/' directory
image_paths = [os.path.join('./assets/', img) for img in os.listdir('./assets/') if img.endswith('.jpg')]

# Process images
net.eval()  # Switch to evaluation mode
with torch.no_grad():  # No need to calculate gradients during evaluation
    for image_path in image_paths:
        image = load_image(image_path)
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f"Prediction for {image_path}: {class_names[predicted.item()]}")